In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error # 平均二乗誤差
from sklearn.metrics import mean_squared_log_error # 対数平均二乗誤差
from sklearn.metrics import r2_score # 決定係数
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

# 1. Import data

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/test.csv")

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.info()

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
test.head()

In [ ]:
test.describe()

'deg_C' has minus values.

# 2. EDA

In [ ]:
train['date_time_2'] = pd.to_datetime(train['date_time'])
#train['year'] = train['date_time_2'].dt.year
train['month'] = train['date_time_2'].dt.month
train['dayofweek'] = train['date_time_2'].dt.dayofweek
train['hour'] = train['date_time_2'].dt.hour

test['date_time_2'] = pd.to_datetime(test['date_time'])
#test['year'] = test['date_time_2'].dt.year
test['month'] = test['date_time_2'].dt.month
test['dayofweek'] = test['date_time_2'].dt.dayofweek
test['hour'] = test['date_time_2'].dt.hour

In [ ]:

sns.catplot(x="month", y="target_carbon_monoxide", data=train, kind="violin")
plt.show()
sns.catplot(x="month", y="target_benzene", data=train, kind="violin")
plt.show()
sns.catplot(x="month", y="target_nitrogen_oxides", data=train, kind="violin")
plt.show()


There are seasonal factors.

In [ ]:

sns.catplot(x="dayofweek", y="target_carbon_monoxide", data=train, kind="violin")
plt.show()
sns.catplot(x="dayofweek", y="target_benzene", data=train, kind="violin")
plt.show()
sns.catplot(x="dayofweek", y="target_nitrogen_oxides", data=train, kind="violin")
plt.show()


All targets are high on weekdays and low on weekends.

In [ ]:

sns.catplot(x="hour", y="target_carbon_monoxide", data=train, kind="violin")
plt.show()
sns.catplot(x="hour", y="target_benzene", data=train, kind="violin")
plt.show()
sns.catplot(x="hour", y="target_nitrogen_oxides", data=train, kind="violin")
plt.show()


There are high values in the commuting time zone.

In [ ]:

sns.pairplot(train[["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5", "deg_C"]])
print("deg_C and sensors(Train)")
plt.show()


In [ ]:

sns.pairplot(test[["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5", "deg_C"]])
print("deg_C and sensors(Test)")
plt.show()


There are strange distributions around 25°C.

In [ ]:

sns.pairplot(train[["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5", "relative_humidity"]])
print("relative_humidity and sensors(Train)")
plt.show()


In [ ]:

sns.pairplot(test[["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5", "relative_humidity"]])
print("relative_humidity and sensors(Test)")
plt.show()


In [ ]:

sns.pairplot(train[["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5", "absolute_humidity"]])
print("absolute_humidity and sensors(Train)")
plt.show()


In [ ]:

sns.pairplot(test[["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5", "absolute_humidity"]])
print("absolute_humidity and sensors(Test)")
plt.show()


In [ ]:
# Relationship between absolute_humidity and sensors

sns.jointplot(x="sensor_1", y="absolute_humidity", data=train)
plt.title("Train")
plt.show()
sns.jointplot(x="sensor_1", y="absolute_humidity", data=test)
plt.title("Test")
plt.show()
sns.jointplot(x="sensor_2", y="absolute_humidity", data=train)
plt.title("Train")
plt.show()
sns.jointplot(x="sensor_2", y="absolute_humidity", data=test)
plt.title("Test")
plt.show()
sns.jointplot(x="sensor_3", y="absolute_humidity", data=train)
plt.title("Train")
plt.show()
sns.jointplot(x="sensor_3", y="absolute_humidity", data=test)
plt.title("Test")
plt.show()
sns.jointplot(x="sensor_4", y="absolute_humidity", data=train)
plt.title("Train")
plt.show()
sns.jointplot(x="sensor_4", y="absolute_humidity", data=test)
plt.title("Test")
plt.show()
sns.jointplot(x="sensor_5", y="absolute_humidity", data=train)
plt.title("Train")
plt.show()
sns.jointplot(x="sensor_5", y="absolute_humidity", data=test)
plt.title("Test")
plt.show()


In [ ]:

sns.pairplot(train[["deg_C", "relative_humidity", "absolute_humidity"]])
print("Temperature and humidity(Train)")
plt.show()


In [ ]:

sns.pairplot(test[["deg_C", "relative_humidity", "absolute_humidity"]])
print("Temperature and humidity(Test)")
plt.show()


# 3. Preprosessing

In [ ]:
%%time

# Seasonal factors
ary =[]
for i in range(len(train["month"])):
    if(train["month"][i] == 1 or train["month"][i] == 2 or train["month"][i] == 12):
        ary.append(1)
    else:
        ary.append(0)
train["is_winter"] = ary

ary =[]
for i in range(len(test["month"])):
    if(test["month"][i] == 1 or test["month"][i] == 2 or test["month"][i] == 12):
        ary.append(1)
    else:
        ary.append(0)
test["is_winter"] = ary

ary =[]
for i in range(len(train["month"])):
    if(train["month"][i] == 3 or train["month"][i] == 4 or train["month"][i] == 5):
        ary.append(1)
    else:
        ary.append(0)
train["is_spring"] = ary

ary =[]
for i in range(len(test["month"])):
    if(test["month"][i] == 3 or test["month"][i] == 4 or test["month"][i] == 5):
        ary.append(1)
    else:
        ary.append(0)
test["is_spring"] = ary

ary =[]
for i in range(len(train["month"])):
    if(train["month"][i] == 6 or train["month"][i] == 7 or train["month"][i] == 8):
        ary.append(1)
    else:
        ary.append(0)
train["is_summer"] = ary

ary =[]
for i in range(len(test["month"])):
    if(test["month"][i] == 6 or test["month"][i] == 7 or test["month"][i] == 8):
        ary.append(1)
    else:
        ary.append(0)
test["is_summer"] = ary

ary =[]
for i in range(len(train["month"])):
    if(train["month"][i] == 9 or train["month"][i] == 10 or train["month"][i] == 11):
        ary.append(1)
    else:
        ary.append(0)
train["is_autumn"] = ary

ary =[]
for i in range(len(test["month"])):
    if(test["month"][i] == 9 or test["month"][i] == 10 or test["month"][i] == 11):
        ary.append(1)
    else:
        ary.append(0)
test["is_autumn"] = ary

# Week factors
train["is_weekend"] = (train["date_time_2"].dt.dayofweek >= 5).astype("int")
test["is_weekend"] = (test["date_time_2"].dt.dayofweek >= 5).astype("int")

# Time zone factors
ary =[]
for i in range(len(train["date_time_2"])):
    if (7 <= train["hour"][i] <= 9):#7,8,9
        ary.append(1)
    else:
        ary.append(0)
train["is_commute_m"] = ary

ary =[]
for i in range(len(test["date_time_2"])):
    if (7 <= test["hour"][i] <= 9):
        ary.append(1)
    else:
        ary.append(0)
test["is_commute_m"] = ary

ary =[]
for i in range(len(train["date_time_2"])):
    if (18 <= train["hour"][i] <= 20):#18,19,20
        ary.append(1)
    else:
        ary.append(0)
train["is_commute_e"] = ary

ary =[]
for i in range(len(test["date_time_2"])):
    if (18 <= test["hour"][i] <= 20):
        ary.append(1)
    else:
        ary.append(0)
test["is_commute_e"] = ary

ary =[]
for i in range(len(train["date_time_2"])):
    if (10 <= train["hour"][i] <= 16):#10～16
        ary.append(1)
    else:
        ary.append(0)
train["is_work"] = ary

ary =[]
for i in range(len(test["date_time_2"])):
    if (10 <= test["hour"][i] <= 16):
        ary.append(1)
    else:
        ary.append(0)
test["is_work"] = ary

ary =[]
for i in range(len(train["date_time_2"])):
    if (train["hour"][i] >= 23 or train["hour"][i] <= 5):#23,5
        ary.append(1)
    else:
        ary.append(0)
train["is_night"] = ary

ary =[]
for i in range(len(test["date_time_2"])):
    if (test["hour"][i] >= 23 or test["hour"][i] <= 5):
        ary.append(1)
    else:
        ary.append(0)
test["is_night"] = ary

# Outlier flag
"""
# deg_C = 23 ～ 25 factors
ary =[]
for i in range(len(train["deg_C"])):
    if (train["deg_C"][i] >= 23 and train["deg_C"][i] < 25):
        ary.append(1)
    else:
        ary.append(0)
train["is_deg_C_23_25"] = ary

ary =[]
for i in range(len(test["deg_C"])):
    if (test["deg_C"][i] >= 23 and test["deg_C"][i] < 25):
        ary.append(1)
    else:
        ary.append(0)
test["is_deg_C_23_25"] = ary
"""
ary =[]
for i in range(len(train["absolute_humidity"])):
    if (train["absolute_humidity"][i] < 0.25 and 23 < train["deg_C"][i] < 25):
        ary.append(1)
    else:
        ary.append(0)
train["is_low_absolute_humidity"] = ary

ary =[]
for i in range(len(test["absolute_humidity"])):
    if (test["absolute_humidity"][i] < 0.25 and 23 < test["deg_C"][i] < 25):
        ary.append(1)
    else:
        ary.append(0)
test["is_low_absolute_humidity"] = ary


In [ ]:
# Concat train and test
all = pd.concat([train,test],ignore_index=True)

# Scaling
month = all['month']
month = np.array(month)
all['month'] = preprocessing.minmax_scale(month[:])
dayofweek = all['dayofweek']
dayofweek = np.array(dayofweek)
all['dayofweek'] = preprocessing.minmax_scale(dayofweek[:])
hour = all['hour']
hour = np.array(hour)
all['hour'] = preprocessing.minmax_scale(hour[:])

deg_C = all['deg_C']
deg_C = np.array(deg_C)
all['deg_C_2'] = preprocessing.minmax_scale(deg_C[:])
relative_humidity = all['relative_humidity']
relative_humidity = np.array(relative_humidity)
all['relative_humidity_2'] = preprocessing.minmax_scale(relative_humidity[:])
absolute_humidity = all['absolute_humidity']
absolute_humidity = np.array(absolute_humidity)
all['absolute_humidity_2'] = preprocessing.minmax_scale(absolute_humidity[:])
sensor_1 = all['sensor_1']
sensor_1 = np.array(sensor_1)
all['sensor_1_2'] = preprocessing.minmax_scale(sensor_1[:])
sensor_2 = all['sensor_2']
sensor_2 = np.array(sensor_2)
all['sensor_2_2'] = preprocessing.minmax_scale(sensor_2[:])
sensor_3 = all['sensor_3']
sensor_3 = np.array(sensor_3)
all['sensor_3_2'] = preprocessing.minmax_scale(sensor_3[:])
sensor_4 = all['sensor_4']
sensor_4 = np.array(sensor_4)
all['sensor_4_2'] = preprocessing.minmax_scale(sensor_4[:])
sensor_5 = all['sensor_5']
sensor_5 = np.array(sensor_5)
all['sensor_5_2'] = preprocessing.minmax_scale(sensor_5[:])

# Split all for train and test
train_scale = all.iloc[train.index[0]:train.index[-1]+1].drop(columns=["deg_C", "relative_humidity", "absolute_humidity", "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"])
test_scale = all.iloc[train.index[-1]+1:].drop(columns=["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides", "deg_C", "relative_humidity", "absolute_humidity", "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"])
test_scale = pd.DataFrame.reset_index(test_scale).drop(columns=["index"])

In [ ]:
train_scale.head()

In [ ]:
test_scale.head()

In [ ]:

%%time
corr = train_scale.corr().round(2)
plt.figure(figsize=(20,10))
sns.heatmap(corr, vmin=-1, vmax=1, center=0, square=False, annot=True, cmap='coolwarm')
plt.show()


There is a high positive correlation between each other's targets.<br>
There is a difference in the correlation between the target and sensors.<br>
Therefore, the prediction is made in descending order of correlation with the sensor.<br><br>
First, I predict the value of benzene only.<br>
Then, after adding the predicted bebzen value as an explanatory variable, I predict the carbon monoxide value only.<br>
Finally, add the predicted carbon monoxide value and then I predict the nitrogen oxides value.

# 4. Modeling and prediction

### 4-1 Modeling for target_benzene

In [ ]:
columns_2 = test_scale.columns.drop(['date_time', 'date_time_2', 'month'])#, 'dayofweek', 'hour'
columns_2

In [ ]:
X = train_scale[columns_2].values
X_test = test_scale[columns_2].values

value_2 = train_scale['target_benzene'].values.reshape(-1,1)

In [ ]:
X_train_2, X_test_2, t_train_2, t_test_2 = train_test_split(X, value_2,test_size=0.3, random_state=0)

lgb_train_2 = lgb.Dataset(X_train_2, t_train_2)
lgb_eval_2 = lgb.Dataset(X_test_2, t_test_2, reference=lgb_train_2)

params_2 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'l2'},
        'learning_rate': 0.002,
        'num_leaves': 16,#16(RMSLE_2 : 0.08844484794714792,R2_2 : 0.978516414910062)
        'num_iterations': 20000,
        'verbosity': -1
}

model_2 = lgb.train(
    params_2,
    train_set=lgb_train_2,
    valid_sets=lgb_eval_2,
    early_stopping_rounds=100,
    verbose_eval=1000
)


# Verification

pred_2 = model_2.predict(X_test_2)

msle_2 = mean_squared_log_error(t_test_2, pred_2) # MSLE(対数平均二乗誤差)の算出
rmsle_2 = np.sqrt(msle_2) # √MSLEの算出
print('RMSLE_2 : {}'.format(rmsle_2))

r2_2 = r2_score(t_test_2,pred_2)
print('R2_2    : {}'.format(r2_2))


### 4-2 Predict target_benzene and next modeling for target_carbon_monoxide

In [ ]:
sample_submission['target_benzene'] = model_2.predict(X_test)

test_scale['target_benzene'] = model_2.predict(X_test)

# Concat train and test
all = pd.concat([train_scale,test_scale],ignore_index=True)
target_benzene = all['target_benzene']
target_benzene = np.array(target_benzene)
all['target_benzene'] = preprocessing.minmax_scale(target_benzene[:])

# Split all for train and test
train_scale = all.iloc[train.index[0]:train.index[-1]+1]
test_scale = all.iloc[train.index[-1]+1:].drop(columns=["target_carbon_monoxide", "target_nitrogen_oxides"])

columns_1 = test_scale.columns.drop(['date_time', 'date_time_2', 'month'])#, 'dayofweek', 'hour'
columns_1

In [ ]:
X = train_scale[columns_1].values
X_test = test_scale[columns_1].values
value_1 = train_scale['target_carbon_monoxide'].values.reshape(-1,1)

In [ ]:
X_train_1, X_test_1, t_train_1, t_test_1 = train_test_split(X, value_1,test_size=0.3, random_state=0)

lgb_train_1 = lgb.Dataset(X_train_1, t_train_1)
lgb_eval_1 = lgb.Dataset(X_test_1, t_test_1, reference=lgb_train_1)

params_1 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'l2'},
        'learning_rate': 0.01,
        'num_leaves': 8,#8(RMSLE_1 : 0.10294909285376616,R2_1 : 0.9311012615666501)
        'num_iterations': 20000,
        'verbosity': -1
}

model_1 = lgb.train(
    params_1,
    train_set=lgb_train_1,
    valid_sets=lgb_eval_1,
    early_stopping_rounds=100,
    verbose_eval=1000
)

# Verification

pred_1 = model_1.predict(X_test_1)

msle_1 = mean_squared_log_error(t_test_1, pred_1)
rmsle_1 = np.sqrt(msle_1)
print('RMSLE_1 : {}'.format(rmsle_1))

r2_1 = r2_score(t_test_1,pred_1)
print('R2_1    : {}'.format(r2_1))

### 4-3 Predict target_carbon_monoxide and next modeling for target_nitrogen_oxides

In [ ]:
sample_submission['target_carbon_monoxide'] = model_1.predict(X_test)

test_scale['target_carbon_monoxide'] = model_1.predict(X_test)

# Concat train and test
all = pd.concat([train_scale,test_scale],ignore_index=True)
target_carbon_monoxide = all['target_carbon_monoxide']
target_carbon_monoxide = np.array(target_carbon_monoxide)
all['target_carbon_monoxide'] = preprocessing.minmax_scale(target_carbon_monoxide[:])

# Split all for train and test
train_scale = all.iloc[train.index[0]:train.index[-1]+1]
test_scale = all.iloc[train.index[-1]+1:].drop(columns=["target_nitrogen_oxides"])

columns_3 = test_scale.columns.drop(['date_time', 'date_time_2', 'month'])
columns_3

In [ ]:
X = train_scale[columns_3].values
X_test = test_scale[columns_3].values
value_3 = train_scale['target_nitrogen_oxides'].values.reshape(-1,1)

In [ ]:

X_train_3, X_test_3, t_train_3, t_test_3 = train_test_split(X, value_3,test_size=0.3, random_state=0)

lgb_train_3 = lgb.Dataset(X_train_3, t_train_3)
lgb_eval_3 = lgb.Dataset(X_test_3, t_test_3, reference=lgb_train_3)

params_3 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'l2'},
        'learning_rate': 0.0005,
        'num_leaves': 92,#132(RMSLE_3 : 0.19056726479718006,R2_3 : 0.9450136429424264)
        'num_iterations': 20000,
        'verbosity': -1
}

model_3 = lgb.train(
    params_3,
    train_set=lgb_train_3,
    valid_sets=lgb_eval_3,
    early_stopping_rounds=100,
    verbose_eval=1000
)


# Verification

pred_3 = model_3.predict(X_test_3)

msle_3 = mean_squared_log_error(t_test_3, pred_3)
rmsle_3 = np.sqrt(msle_3)
print('RMSLE_3 : {}'.format(rmsle_3))

r2_3 = r2_score(t_test_3,pred_3)
print('R2_3    : {}'.format(r2_3))

### 4-4 Predict target_nitrogen_oxides

In [ ]:
sample_submission['target_nitrogen_oxides'] = model_3.predict(X_test)

sample_submission

# 5. Make submission file

In [ ]:
sample_submission.to_csv('submission.csv', index=False)